In [1]:
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image[512:,512:], dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [2]:
model.plot()

In [3]:
1/0.24927006

4.011713239849183

In [2]:
model.find_peaks()
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20,a_limit=50, b_limit=50 )

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


Updated peak locations.
Origin selected: [717. 860.]
Vector a selected: [24.  0.]
Vector b selected: [  1. -24.]
In pixel: Origin: [717. 860.], Vector a: [24.  0.], Vector b: [  1. -24.]
In space: Origin: [117.97727572 141.50691369] A, Vector a: [3.94903015 0.        ] A, Vector b: [ 0.16454292 -3.94903015] A
Origin selected: [768 768]
Vector a selected: [63  0]
Vector b selected: [ 0 63]
In pixel: Origin: [768 768], Vector a: [63  0], Vector b: [ 0 63]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [12]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-4, plot=False, step_size=0.01,verbose=False)

Fitting random batch:   0%|          | 0/4 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [4]:
model.plot_fitting()

In [4]:
from qem.crystal_analyzer import CrystalAnalyzer
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image[512:1500,512:1500], dx=dx,units='A')
model.find_peaks()
crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'])

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


Updated peak locations.


In [5]:

cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.import_crystal_structure(cif_file)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

fft_image = np.fft.fftshift(np.fft.fft2(model.image))
plt.imshow(np.abs(fft_image),norm=LogNorm())
plt.colorbar()

from skimage.feature.peak import peak_local_max

peaks = peak_local_max(np.abs(fft_image), min_distance=20,threshold_abs=10*np.mean(np.abs(fft_image)))
plt.scatter(peaks[:,1],peaks[:,0],c='r')


In [3]:
from qem.gui_classes import InteractivePlot
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
from skimage.feature import peak_local_max

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')
fft_image = np.fft.fftshift(np.fft.fft2(model.image))
peaks = peak_local_max(np.abs(fft_image), min_distance=20,threshold_abs=10*np.mean(np.abs(fft_image)))

plot = InteractivePlot(np.log(np.abs(fft_image)),peaks,dx = dx, units='1/A')
plot.select_vectors()

ValueError: Invalid unit (1/A) with dimension

In [27]:
np.mean(np.abs(fft_image))

63450.00874856338

In [3]:
crystal_analyzer.choose_lattice_vectors()

Origin selected: [547. 592.]
Vector a selected: [26. -2.]
Vector b selected: [  1. -26.]
Origin: [547. 592.], Vector a: [26. -2.], Vector b: [  1. -26.]


(array([547., 592.]), array([26., -2.]), array([  1., -26.]))

In [4]:
crystal_analyzer._origin_adaptive = None
shift_center = crystal_analyzer.shift_origin_adaptive(50, 50)
plt.imshow(model.image, cmap='gray')
peaks = []
for key, value in shift_center.items():
    peaks.append(value)

plt.scatter(np.array(peaks)[:,1], np.array(peaks)[:,0], c='r', s=10)
plt.show()

In [23]:
im_refined = model.predict_local(params=model.params)
plt.imshow(im_refined)

In [25]:
plt.imshow(model.image-im_refined)

In [18]:
model.atom_types

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,